In [ ]:
!pip install -upgrade -quiet langchain-core langchain-community langchain-openai
!pip install openai
!pip install langchain
!pip install langchain_openai

In [ ]:
#튜플이나 리스트같은 '컬렉션타입' 데이터에서 원하는 인덱스의 요소를 추출하는 데 사용
from operator import itemgetter

#러너블 클래 : 함수를 감싸서 체인화 할 수 있게 해 줌
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

#Output(결과물)을 content만 가져올 수 있도록 변환
# choices[0].content... <- {model,,, }
#질의응답 중 응답내용만 가져오도록 정리해주는 함수
from langchain_core.output_parsers import StrOutputParser

#특정한 대화 양식(템플릿)을 정의하여 쉽게 질의응답 할 수 있도록 해줌
#우리가 나눈 대화를 관리하는 데 사용
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

#openai와의 대화형 질의응답
from langchain_openai import ChatOpenAI

#openai와 나눈 대화를 저장하고 관리하는데 사용하는 클래스
#"이전 대화를 기반으로" 현재의 대화를 이끌어나가도록 함
#히스토리 관리해줌-memory
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
#랭체인 기반 챗봇을 만들기 위해서 필요한 라이브러리 기능?
#인덱스 요소 추출-?, 러너블 함수-chain을 편하게 이용(내가 만든 함수를 끼워넣을 수 있음),대화형AI,응답내용만 가져옴(=컨텐츠 내용만 뽑아오기),메모리 저장 기능

In [ ]:
import os
import openai

#이전의 방식은 소스코드를 내가 업로드 했을 떄, 다른 사람들이 내 키를 볼 수 있음
#'이름'==userdata.get(이름),값=내 open_api_key
from google.colab import userdata
os.environ['OPENAI_API_KEY']=userdata.get('OPEN_AI_KEY') #('OPEN_AI_KEY')은 변경 가능

###챗봇을 정의해보자.


In [ ]:
#맥락 정의하기
system_prompt='''
너는 나랑 친구인 상담사야.
내가 물어보는 질문에 대해서 친절하게 잘 알져줘야 해
한국어로 답변해줘.
'''

In [ ]:
chat_prompt_template=ChatPromptTemplate.from_messages([
    ('system',system_prompt),
    #우리의 과거 대화 데이터 기억
    MessagesPlaceholder(variable_name='history'),
     ('human','{user_input}')
])

#k=기억할 대화 쌍의 개수(보낸 메시지가 아니라 대화 쌍의 수)
#총 3개의 이전 대화 쌍을 기억함.
memory=ConversationBufferWindowMemory(k=3,return_messages=True)

In [ ]:
chat_model=ChatOpenAI()
output_parser=StrOutputParser()

In [ ]:
#우리가 한 데이터를 받아서 chain으로 돌린후 저장
chain=({'user_input':RunnablePassthrough()} |
       RunnablePassthrough.assign(history=RunnableLambda(memory.load_memory_variables) | itemgetter('history'))
       |chat_prompt_template
       |chat_model
       |output_parser)

#입력값을 그대도 전달-> 과거 데이터가 저장될 history 변수에 람다 함수의 결과가 저장됨->
#프롬프트에 넣기->모델에 넣고->결과 출력

In [ ]:
memory.load_memory_variables({})

In [ ]:
def load_memory(_):
    return memory.load_memory_variables(_)['history']

In [ ]:
#받은 메세지를 체인으로 변환 후
#메모리에 저장 후 받기
def chat_with_user(user_message):
    ai_message=chain.invoke(user_message) #답변

    memory.save_context({'input':user_message},{'output':ai_message})
    print(memory.load_memory_variables({})) #메모리 저장 내용 출력

    return ai_message

In [ ]:
chat_with_user("안녕 오늘 비온다고 하는데 너 우산 챙겼니?")

In [ ]:
chat_with_user("안녕 오늘 하루는 어땠어?")

In [ ]:
while True:
    user_message=input('USER>>')
    if user_message == 'quit':
        break
    ai_message=chat_with_user(user_message)
    print(f'AI>>{ai_message}')

##그라디오를 활용한 챗봇 껍데기 만들기

In [ ]:
#내가 만든 코드를 동일하게 보여주는 사이트가 임시로 생성됨

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

import random
import time


def respond(user_message,chat_view):
    #채팅을 주면 그걸 받아옴
    ai_message=chat_with_user(user_message)
    chat_view.append((user_message,ai_message))

    time.sleep(1)

    return '',chat_view

with gr.Blocks() as demo:
    #창을 정의
    chat_view=gr.Chatbot(label="채팅창")
    user_textbox=gr.Textbox(label="입력")

    #입력한 내용을 갖고 출력해라
    user_textbox.submit(respond,[user_textbox,chat_view],[user_textbox, chat_view])

#그라디오를 demo라는 이름 컨트롤하게 묶어줌
#share=True ->링크를 만들건지 확인하기
demo.launch(share=True)